# COGS 108 - EDA Checkpoint

# Names

- Crystal Zhan
- Akil Selvan Rajendra Janarthanan 
- Kristen Prescaro
- Kristine Thipatima
- Ethan Dinh-Luong

<a id='research_question'></a>
# Research Question

How did the addition of the Fairy type Pokemon change competitive battling?

# Setup

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data Cleaning

## Cleaning Informational Data (Pokemon Moves, Pokemon Types)

The list of moves is in a semi-structed JSON file, which we needed to clean up so we only had the relevant information about the moves. 

In [6]:
moves = pd.read_json("Pokedex and Moves/data/moves.json")
moves.head()

,10000000voltthunderbolt,absorb,accelerock,acid,acidarmor,aciddownpour,acidspray,acrobatics,acupressure,aerialace,...,workup,worryseed,wrap,wringout,xscissor,yawn,zapcannon,zenheadbutt,zingzap,zippyzap
num,719,71,709,51,151,628,491,512,367,332,...,526,388,35,378,404,281,192,428,716,729
accuracy,True,100,100,100,True,True,100,100,True,True,...,True,100,90,100,100,True,50,90,100,100
basePower,195,20,40,40,0,1,40,55,0,60,...,0,0,15,0,80,0,120,80,80,80
category,Special,Special,Physical,Special,Status,Physical,Special,Physical,Status,Physical,...,Status,Status,Physical,Special,Physical,Status,Special,Physical,Physical,Physical
isNonstandard,Past,NaN,NaN,NaN,NaN,Past,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Past,NaN,NaN,NaN,NaN,NaN,LGPE


As can be seen, the names of the moves are the column names and the variables are the row names. However, we want it the other way around, with the move names as the index. 

We also only want the move name and the move type, so we remove all other variables. 

Next, we reset the index so there are index numbers and rename the index column to be called move.

Now, our move dataset is clean. 

In [7]:
# switching the column and row names 
moves = moves.T

# removing all columns except move name & type
moves = moves.loc[:,["name", "type"]]

#resetting the index numbers
moves = moves.reset_index()

#renaming index to be called move 
moves.rename(columns = {"index":"move"}, inplace=True)
moves.head()

,move,name,type
0,10000000voltthunderbolt,"10,000,000 Volt Thunderbolt",Electric
1,absorb,Absorb,Grass
2,accelerock,Accelerock,Rock
3,acid,Acid,Poison
4,acidarmor,Acid Armor,Poison


Now, we apply the same cleaning steps with the list of Pokemon. 

However, there is an extra step of removing any Mega or Gmax form Pokemon, as our Pokemon Showdown dataset doesn't include them, nor would they affect the usage of Pokemon based on type. 

In [8]:
pokemon = pd.read_json("Pokedex and Moves/data/pokedex.json")

#takes column name and row name and flips them
pokemon = pokemon.T 
pokemon = pokemon.iloc[0:1155, :]

#deletes all columns except for name and types
pokemon = pokemon.loc[:,["name", "types"]]
pokemon = pokemon.reset_index()

#renames index to be pokemon
pokemon.rename(columns = {"index":"pokemon"}, inplace=True)

#drop rows that contain the partial string "gmax or mega" in the pokemon column
pokemon = pokemon[~pokemon.pokemon.str.contains("gmax")]
pokemon = pokemon[~pokemon.name.str.contains("-Mega")]

pokemon.head()


,pokemon,name,types
0,bulbasaur,Bulbasaur,"[Grass, Poison]"
1,ivysaur,Ivysaur,"[Grass, Poison]"
2,venusaur,Venusaur,"[Grass, Poison]"
5,charmander,Charmander,[Fire]
6,charmeleon,Charmeleon,[Fire]


The next step for the Pokemon list is to split the type column into two parts, as some Pokemon have two types, while others have one. For those with one type, we will put null for the 2nd type. 

In [9]:
type1 = []
type2 = []

#loop through the list of pokemon and puts their types in their own columns
#puts None if there's no secondary type
for x in pokemon["types"]:
    type1.append(x[0])
    if (len(x) == 2):
        type2.append(x[1])
    else:
        type2.append(None)

pokemon["types"] = type1
pokemon["type2"] = type2
pokemon.head()


,pokemon,name,types,type2
0,bulbasaur,Bulbasaur,Grass,Poison
1,ivysaur,Ivysaur,Grass,Poison
2,venusaur,Venusaur,Grass,Poison
5,charmander,Charmander,Fire,None
6,charmeleon,Charmeleon,Fire,None


## Pokemon Showdown Battle Stats

The data given by Pokemon Showdown is several semi-structured JSON format files, where cleaning was needed in every file to read the data into a usable format. 

*The cleaning process exampled below was repeated for all other JSON files.*

Unnecessary data prior to our analysis was removed where data was null.

In [10]:
raw = pd.read_json("Pokemon Usage/September/raw/gen8/gen8ou-0.json")
df = raw[raw['data'].notna()]['data']
df

Mr. Mime-Galar    {'Moves': {'': 32.0, 'healingwish': 226.0, 'bl...
Eevee             {'Moves': {'': 197.0, 'rest': 7.0, 'mudslap': ...
Torracat          {'Moves': {'': 1.0, 'firespin': 20.0, 'leechli...
Poliwrath         {'Moves': {'': 58.0, 'counter': 48.0, 'liquida...
Emolga            {'Moves': {'': 2.0, 'eerieimpulse': 47.0, 'ris...
                                        ...                        
Shedinja          {'Moves': {'': 578.0, 'absorb': 11.0, 'falsesw...
Wishiwashi        {'Moves': {'': 67.0, 'liquidation': 393.0, 'be...
Sneasel           {'Moves': {'counter': 3.0, 'beatup': 9.0, 'bli...
Hitmontop         {'Moves': {'': 208.0, 'detect': 89.0, 'quickgu...
Kingdra           {'Moves': {'': 57.0, 'icywind': 32.0, 'liquida...
Name: data, Length: 440, dtype: object

To narrow down the data desired for our analysis, the following criteria were used to filter out the data:
- Pokemon with at least 2% usage
- Each Pokemon's Top 6 Moves

Additionally, each dataframe includes 4 more columns identifying which JSON file the data originated from, denoted by **Gen**, **Format**, **Rating**, and **Recent** given in the first few rows of the JSON file.

In [11]:
### Dictionary to make the DataFrame
top_mons = {}

### Saves the Pokemon as Indexes
ix = list(df.index)

### For each Observation
for row in range(len(df)):

    ### At least 2% Usage
    if df[row]['usage'] >= .02:

        ### Pokemon Name
        mon = ix[row]

        ### Finds the Top 6 Moves
        top_6 = list(dict(sorted(df[row]['Moves'].items(), key=lambda item: item[1], reverse=True)))[:6]
        
        ### Saves info to dictionary
        top_mons[mon] = [top_6, df[row]['usage']]

### Output DataFrame
cleaned = pd.DataFrame.from_dict(top_mons, orient = 'index').rename(columns = {0:"Moves", 1:"Usage"})

### Format, Gen, Rating Variables
metagame = raw.loc['metagame'][0]
gen = metagame[3]
format_name = metagame[4:]
rating = raw.loc["cutoff"][0]

### Adding the Variables to the DF
cleaned["Gen"] = gen
cleaned["Format"] = format_name
cleaned["Min Rating"] = rating
cleaned.head()


,Moves,Usage,Gen,Format,Min Rating
Landorus-Therian,"[earthquake, uturn, stealthrock, knockoff, tox...",0.304108,8,ou,0.0
Blissey,"[softboiled, seismictoss, toxic, teleport, thu...",0.084829,8,ou,0.0
Slowbro,"[scald, teleport, slackoff, futuresight, icebe...",0.057747,8,ou,0.0
Crawdaunt,"[aquajet, knockoff, crabhammer, swordsdance, c...",0.028303,8,ou,0.0
Urshifu-Rapid-Strike,"[surgingstrikes, closecombat, aquajet, uturn, ...",0.129478,8,ou,0.0


## Joining the Datasets

### Showdown Usage with Moves

Firstly, we imported all the cleaned data from the prior section that was applied to all JSON files.

In [12]:
### Importing Usage DFs and Merging Them
recentdf = pd.read_csv("Pokemon Usage/SeptemberData.csv",index_col=0).rename(columns = {'index':"name"})
olddf =  pd.read_csv("Pokemon Usage/OldData.csv",index_col=0).rename(columns = {'index':"name"})
usagedf = recentdf.merge(olddf, how='outer')
print(usagedf.shape)
usagedf.head()

(5916, 7)


,name,Moves,Usage,Gen,Format,Min Rating,Recent
0,Bronzong,"['gyroball', 'explosion', 'earthquake', 'steal...",0.124514,4.0,ou,0.0,1
1,Swampert,"['earthquake', 'stealthrock', 'waterfall', 'ic...",0.120051,4.0,ou,0.0,1
2,Snorlax,"['bodyslam', 'crunch', 'earthquake', 'curse', ...",0.040116,4.0,ou,0.0,1
3,Uxie,"['stealthrock', 'uturn', 'thunderwave', 'psych...",0.023561,4.0,ou,0.0,1
4,Machamp,"['dynamicpunch', 'payback', 'bulletpunch', 'ic...",0.102777,4.0,ou,0.0,1


The **Moves** column deceptively looks cleaned in a list format, but upon closer observation, it is actually a *string* representation of a list.

In [13]:
type(usagedf["Moves"][0])

str

Addressing this, we will convert the string into a list by stripping the list representation and splitting the string by commas. Afterwards, we can proceed with standardizing the moves by its type.

In [14]:
### Importing Moves DF
movesdf = pd.read_csv("Pokedex and Moves/moves.csv", index_col=0).set_index('move')

### Standardize the Moves
def standardize_moves(df):

    ### List Representations of each Column
    type1 = []
    type2 = []
    type3 = []
    type4 = []
    type5 = []
    type6 = []

    ### List of Pointers to Column Lists
    type_list = [type1, type2, type3, type4, type5, type6]

    ### For Each Row
    for row in range(len(df["Moves"])):

        ### Strip the List Borders and Create a List by Splitting on Commas in the String
        moves = df["Moves"][row].strip('][').split(', ')

        ### Tracks which Column Each Move Belongs to
        list_num = 0

        ### For Each Column
        for each in range(6):

            ### If a Pokemon has less than 6 moves, return nan
            if each >= len(moves):
                move_type = np.nan
                type_list[list_num].append(move_type)
                list_num += 1

            else:
                ### Removes Outer Apostrophes, if any
                string = moves[each].strip("'")

                ### If a Popular Move is an Empty Slot, return nan
                if (string) == "":
                    move_type = np.nan
                    type_list[list_num].append(move_type)
                    list_num += 1
                else:

                    ### Find Move in Move DF containing Type
                    move_type = movesdf.loc[string]['type']

                    ### Add to Column
                    type_list[list_num].append(move_type)

                    ### Change Column
                    list_num += 1

    ### Piecing Everything Together
    df["Type 1"] = type_list[0]
    df["Type 2"] = type_list[1]
    df["Type 3"] = type_list[2]
    df["Type 4"] = type_list[3]
    df["Type 5"] = type_list[4]
    df["Type 6"] = type_list[5]

    return df

usagemovesdf = standardize_moves(usagedf)
usagemovesdf.head()

,name,Moves,Usage,Gen,Format,Min Rating,Recent,Type 1,Type 2,Type 3,Type 4,Type 5,Type 6
0,Bronzong,"['gyroball', 'explosion', 'earthquake', 'steal...",0.124514,4.0,ou,0.0,1,Steel,Normal,Ground,Rock,Psychic,Water
1,Swampert,"['earthquake', 'stealthrock', 'waterfall', 'ic...",0.120051,4.0,ou,0.0,1,Ground,Rock,Water,Ice,Ice,Normal
2,Snorlax,"['bodyslam', 'crunch', 'earthquake', 'curse', ...",0.040116,4.0,ou,0.0,1,Normal,Dark,Ground,Ghost,Psychic,Fire
3,Uxie,"['stealthrock', 'uturn', 'thunderwave', 'psych...",0.023561,4.0,ou,0.0,1,Rock,Bug,Electric,Psychic,Psychic,Psychic
4,Machamp,"['dynamicpunch', 'payback', 'bulletpunch', 'ic...",0.102777,4.0,ou,0.0,1,Fighting,Dark,Steel,Ice,Rock,Normal


### Showdown Usage with Pokemon Types

As opposed to the complexity of joining moves with usage, simply joining the Pokemon Types DF stats on the Showdown Usage on **name** will add two new columns **Type 1** and **Type 2** to the Showdown Usage DF.

In [27]:
### Import Usage Data
usagedf = recentdf.merge(olddf, how='outer')

### Import Pokemon Types Data
pokedexdf = pd.read_csv("Pokedex and Moves/pokemon.csv", index_col=0).set_index('name')[['types','type2']]

### Left Join Usage DF with Pokemon Types DF on "name" while renaming the additional columns
usagetypesdf = usagedf.join(pokedexdf, on="name").rename(columns={'types': "Type 1", "type2": "Type 2"})
usagetypesdf

,name,Moves,Usage,Gen,Format,Min Rating,Recent,Type 1,Type 2
0,Bronzong,"['gyroball', 'explosion', 'earthquake', 'steal...",0.124514,4.0,ou,0.0,1,Steel,Psychic
1,Swampert,"['earthquake', 'stealthrock', 'waterfall', 'ic...",0.120051,4.0,ou,0.0,1,Water,Ground
2,Snorlax,"['bodyslam', 'crunch', 'earthquake', 'curse', ...",0.040116,4.0,ou,0.0,1,Normal,NaN
3,Uxie,"['stealthrock', 'uturn', 'thunderwave', 'psych...",0.023561,4.0,ou,0.0,1,Psychic,NaN
4,Machamp,"['dynamicpunch', 'payback', 'bulletpunch', 'ic...",0.102777,4.0,ou,0.0,1,Fighting,NaN
...,...,...,...,...,...,...,...,...,...
5911,Charizard,"['solarbeam', 'fireblast', 'hurricane', 'airsl...",0.030482,8.0,ubers,0.0,0,Fire,Flying
5912,Shuckle,"['stickyweb', 'stealthrock', 'encore', 'finalg...",0.045773,8.0,ubers,0.0,0,Bug,Rock
5913,Hatterene,"['mysticalfire', 'psychic', 'dazzlinggleam', '...",0.026064,8.0,ubers,0.0,0,Psychic,Fairy
5914,Zacian-Crowned,"['behemothblade', 'playrough', 'swordsdance', ...",0.921806,8.0,ubers,0.0,0,Fairy,Steel


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [16]:
def SE_moves(move_type, df, col):
    if move_type == "Dragon":
        SE_moves = ["Fairy","Ice","Dragon"]
    else:
        SE_moves = ["Steel","Poison"]

    is_SE = None
    for each in SE_moves:
        if is_SE is None:
            is_SE = (df[col] == each)*1
        else:
            is_SE += (df[col] == each)*1
    
    return is_SE


def find_all_SE(df):
    types = ["Dragon", "Fairy"]
    col = ["Type 1", "Type 2", "Type 3", "Type 4", "Type 5", "Type 6"]
    SE_col = ["SE Dragon", "SE Fairy"]

    for each in types:

        all_SE = None

        for col_name in col:

            if all_SE is None:
                all_SE = SE_moves(each, df, col_name)
            else:
                all_SE += SE_moves(each, df, col_name)

        ### Uncomment to change to binary results 
        # all_SE = (all_SE > 0) * 1
        df[SE_col[0]] = all_SE
        SE_col = SE_col[1:]
    
    return df

usagemovesdf = find_all_SE(usagemovesdf)
usagemovesdf.head()

,name,Moves,Usage,Gen,Format,Min Rating,Recent,Type 1,Type 2,Type 3,Type 4,Type 5,Type 6,SE Dragon,SE Fairy
0,Bronzong,"['gyroball', 'explosion', 'earthquake', 'steal...",0.124514,4.0,ou,0.0,1,Steel,Normal,Ground,Rock,Psychic,Water,0,1
1,Swampert,"['earthquake', 'stealthrock', 'waterfall', 'ic...",0.120051,4.0,ou,0.0,1,Ground,Rock,Water,Ice,Ice,Normal,2,0
2,Snorlax,"['bodyslam', 'crunch', 'earthquake', 'curse', ...",0.040116,4.0,ou,0.0,1,Normal,Dark,Ground,Ghost,Psychic,Fire,0,0
3,Uxie,"['stealthrock', 'uturn', 'thunderwave', 'psych...",0.023561,4.0,ou,0.0,1,Rock,Bug,Electric,Psychic,Psychic,Psychic,0,0
4,Machamp,"['dynamicpunch', 'payback', 'bulletpunch', 'ic...",0.102777,4.0,ou,0.0,1,Fighting,Dark,Steel,Ice,Rock,Normal,1,1


In [74]:

test = usagetypesdf[(usagetypesdf['Type 1'] == 'Dragon') | (usagetypesdf['Type 2'] == 'Dragon')]

test2 = usagetypesdf[(usagetypesdf['Type 1'] != 'Dragon') & (usagetypesdf['Type 2'] != 'Dragon')]

#########################################3


ubersusage = usagetypesdf[(usagetypesdf["Format"] == "ubers")]

dubersusage = ubersusage[(ubersusage['Type 1'] == 'Dragon') | (ubersusage['Type 2'] == 'Dragon')]
nuberusage = ubersusage[(ubersusage['Type 1'] != 'Dragon') & (ubersusage['Type 2'] != 'Dragon')]


###############################################
#take name, usage, and make a new column that says if they're dragon or not dragon
ubersusage = dubersusage[["name", "Usage"]]
uberusage["type"] = "Dragon"

dragon = usagetypesdf.loc[((usagetypesdf['Type 1']=='Dragon') | (usagetypesdf['Type 2']=='Dragon')), :]
dragon['Usage'].sum()





duberusage4 = test[(test["Gen"] == 4)]
nuberusage4 = test2[(test2["Gen"] == 4)]

#print(nuberusage4["Usage"].sum())
#duberusage4["Usage"].sum()

#sns.countplot(y = 'Type', data = duberusage4);

duberusage4



,name,Moves,Usage,Gen,Format,Min Rating,Recent,Type 1,Type 2
9,Latias,"['dracometeor', 'surf', 'dragonpulse', 'trick'...",0.183768,4.0,ou,0.0,1,Dragon,Psychic
28,Dragonite,"['extremespeed', 'outrage', 'dragondance', 'ea...",0.165246,4.0,ou,0.0,1,Dragon,Flying
48,Flygon,"['earthquake', 'uturn', 'outrage', 'thunderpun...",0.161407,4.0,ou,0.0,1,Ground,Dragon
54,Kingdra,"['waterfall', 'outrage', 'dragondance', 'hydro...",0.092220,4.0,ou,0.0,1,Water,Dragon
64,Latias,"['dracometeor', 'surf', 'dragonpulse', 'trick'...",0.203363,4.0,ou,1500.0,1,Dragon,Psychic
...,...,...,...,...,...,...,...,...,...
2856,Dragonite,"['extremespeed', 'earthquake', 'dracometeor', ...",0.169068,4.0,ou,1630.0,0,Dragon,Flying
2887,Flygon,"['earthquake', 'uturn', 'outrage', 'stoneedge'...",0.234171,4.0,ou,1630.0,0,Ground,Dragon
2915,Kingdra,"['dracometeor', 'hydropump', 'raindance', 'wat...",0.030471,4.0,ou,1630.0,0,Water,Dragon
2922,Flygon,"['earthquake', 'uturn', 'outrage', 'thunderpun...",0.255811,4.0,ou,1760.0,0,Ground,Dragon


In [81]:
ubers = usagetypesdf[(usagetypesdf["Format"] == "ubers") & (usagetypesdf["Gen"] == 4.0)]

dragon = ubers.loc[((ubers['Type 1']=='Dragon') | (ubers['Type 2']=='Dragon')), :]
dragon['Usage'].sum()
dragon["Type"] = "Dragon"

notdragon = ubers.loc[((ubers['Type 1'] != 'Dragon') & (ubers['Type 2'] != 'Dragon')), :]
notdragon['Usage'].sum()
notdragon["Type"] = "notDragon"

dragon

dragon['Usage'].sum()
gen4uber = usagetypesdf[usagetypesdf["Gen"] == 4.0][usagetypesdf["Format"] == 'ubers']
dragon = gen4uber.loc[((gen4uber['Type 1']=='Dragon') | (gen4uber['Type 2']=='Dragon')), :]
dragonusage = dragon['Usage'].sum()
notdragon = gen4uber.loc[((gen4uber['Type 1']!='Dragon') & (gen4uber['Type 2']!='Dragon')), :]
notdragonusage = notdragon['Usage'].sum()

C:\Users\ocari\AppData\Local\Temp/ipykernel_26632/4174702637.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dragon["Type"] = "Dragon"
C:\Users\ocari\AppData\Local\Temp/ipykernel_26632/4174702637.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notdragon["Type"] = "notDragon"


,name,Moves,Usage,Gen,Format,Min Rating,Recent,Type 1,Type 2,Type
188,Rayquaza,"['extremespeed', 'earthquake', 'outrage', 'dra...",0.218509,4.0,ubers,0.0,1,Dragon,Flying,Dragon
189,Garchomp,"['earthquake', 'dragonclaw', 'outrage', 'sword...",0.233980,4.0,ubers,0.0,1,Dragon,Ground,Dragon
191,Latias,"['recover', 'dragonpulse', 'calmmind', 'grassk...",0.096336,4.0,ubers,0.0,1,Dragon,Psychic,Dragon
194,Giratina,"['willowisp', 'rest', 'dragonpulse', 'sleeptal...",0.069469,4.0,ubers,0.0,1,Ghost,Dragon,Dragon
195,Palkia,"['spacialrend', 'thunder', 'surf', 'fireblast'...",0.309520,4.0,ubers,0.0,1,Water,Dragon,Dragon
198,Salamence,"['earthquake', 'dragondance', 'outrage', 'drag...",0.025355,4.0,ubers,0.0,1,Dragon,Flying,Dragon
206,Giratina-Origin,"['shadowsneak', 'dracometeor', 'earthquake', '...",0.485098,4.0,ubers,0.0,1,Ghost,Dragon,Dragon
210,Dialga,"['dracometeor', 'thunder', 'stealthrock', 'dra...",0.522966,4.0,ubers,0.0,1,Steel,Dragon,Dragon
216,Latios,"['dracometeor', 'recover', 'grassknot', 'thund...",0.214433,4.0,ubers,0.0,1,Dragon,Psychic,Dragon
222,Kingdra,"['dracometeor', 'surf', 'outrage', 'dragonpuls...",0.044289,4.0,ubers,0.0,1,Water,Dragon,Dragon
